# Baseline 流程设计

快速搭建一个符合比赛要求的pipeline出来，做好模块划分和设计，方便后面分工协作。

---

**调试**：

1. import更新的调试需要先`restart IPython kernel`

## Question or problem definition (**TODO**)

dataset为三元组集合，$\mathbf{D} = \{p_i, s_i, e_i\}_{i=1}^N$，其中$p_i$为路径，$s_i$是出发时间，$e_i$是到达时间。
给定一个query $q=(o_q, d_q, s_q, \hat{p}_q)$，其中$o_q$是出发地，$d_q$是目的地，$s_q$是出发时间，
$\hat{p}_q$是验证集的给出的一段路径

首先明确是regression，ETA领域主流做法有两种，模拟轨迹或者直接预测时间，baseline这里先考虑直接获得时间.

这里面会遇到很多问题，也是之后要解决的：

1. $\boldsymbol{p}$所代表的一系列特征的选取（不定长的特征提取——初步考虑RNN，很核心的一个问题就是海运是个连续的物理空间，很难表示path）
2. 预处理怎么填充
3. 模型选取（GBDT, FM, ...）

## Startup
整体打算是把大的流程用jupyter写，而模块写成单独的.py，所有jupyter文件放在`experiment`目录下，所以要import别的模块需要首先修改系统目录。

In [1]:
import sys, os
os.chdir('..')
print(os.path.abspath('.'))

e:\Code\ML2020-ETA


接下来就是import一系列需要用的包

In [2]:
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.model_selection import KFold

## Feature Exploration, Engineering and Cleaning
这部分要读取数据，转换成模型输入的数据。
内部可能可以再拆分成不同的小任务，毕竟特征工程这一块在总工作量中所占比重非常高。

所有数据都放在`data`目录下，包括未处理的和处理过的中间结果。

In [3]:
# load data
port = pd.read_csv('data/port.csv')
loading_order_event = pd.read_csv('data/loadingOrderEvent.csv', low_memory=False) # 辅助数据
gps_record = pd.read_csv('data/train_dev.csv', low_memory=False, chunksize=None) # 前期先小规模数据（对于大数据量调整chuncksize参数）
test_data = pd.read_csv('data/A_testData0531.csv', low_memory=False)
# 允许使用外部数据，可能包括补充、修正port坐标的数据，以及不同海域航行特征的数据等等

关于**A轮测试数据**，目前看TRACE似乎都只有起止港，目前就按照这个做，后面如果出现了差别再改动也是来得及的。

In [4]:
# 测试test data里面有哪些起止路线
routes = set()
for _, row in test_data.iterrows():
    if row['TRANSPORT_TRACE'] not in routes:
        routes.add(row['TRANSPORT_TRACE'])
print(routes)

{'CNSHK-SIKOP', 'COBUN-HKHKG', 'CNSHK-PKQCT', 'CNSHK-SGSIN', 'CNYTN-RTM', 'HKHKG-FRFOS', 'CNYTN-CAVAN', 'CNSHK-MYTPP', 'CNSHK-LBBEY', 'CNHKG-MXZLO', 'CNSHK-GRPIR', 'CNYTN-NZAKL', 'CNYTN-MXZLO', 'CNSHK-ZADUR', 'CNSHA-PAMIT', 'CNSHK-ESALG', 'CNSHK-CLVAP', 'CNYTN-MTMLA', 'CNYTN-ARENA', 'CNSHA-SGSIN', 'CNYTN-MATNG', 'CNYTN-PAONX'}


关于特征工程，baseline这里尽量从简，之后慢慢改。

目前对数据特征的一些总结理解。

- 参考一下[正是赛习题讲解](https://bbs.huaweicloud.com/forum/thread-57457-1-1.html)，里面有说历史运单数据和GPS中的运单数据是一一对应的（实际上不是），由loadingOrder关联，但是历史信息由于是手动录入误差可能较大，**建议通过GPS录入**。

- 作为baseline尽量保证训练数据和测试数据特征选取一致，方便理解，但是后面肯定是要在训练里面加入更多的东西（大概）

---

**训练数据**：GPS数据为主，历史运单数据为辅，港口数据<u>可能需要外部补充</u>

> 先主要考察carrierName, timestamp, 

> spatial feature: longtitude, latitude, speed, direction, vesselNextPort, vesselNextPortETA, vesselStatus, TRANSPORT_TRACE

> temporal feature: timestamp

> data reliability: vesselDatasource

关于`vesselStatus`，为AIS船舶航行状态，具体来说为

名称 | id
:-|:-:
under way using engine(动力船在航) | 0
under way sailing(帆船在航) | 1
at anchor(锚泊) | 2
not under command(失控) | 3
moored(系泊) | 4
contrained by her draft(吃水限制) | 5

**测试数据**：loadingOrder, timestamp, longtitude, latitude, speed, direction, carrierName, vesselMMSI, onboardDate, TRANSPORT_TRACE

**预测数据**：loadingOrder, timestamp, longtitude, latitude, carrierName, vesselMMSI, onboardDate, **ETA**, **creatDate**



In [ ]:
# port data
# 索引查找

In [11]:
# 关于GPS数据：每个运单表示一次运输的运输单号，不会重复使用，一次运输过程中的多条GPS数据拥有相同的运输单号。船号为运单货物所在的船编号，会重复出现在不同次运输的GPS数据中

# GPS data
## 时间序列问题的角度（季节？）
## speed补充（就近 or 插值）
## direction补充（就近 or 插值）
## vesselNextport补充
print('nextport na count:', gps_record['vesselNextport'].isnull().sum())
## vesselNextportETA补充
print('nextport eta na count:', gps_record['vesselNextportETA'].isnull().sum())
## vesselStatus修改&补充
from preprocess import vessel_status
gps_record['vesselStatus'].fillna(-1, inplace=True)
gps_record['vesselStatus'].apply(lambda x: -1 if x == -1 else vessel_status[x])
## vesselDatasource补充
from preprocess import vessel_datasource
gps_record['vesselDatasource'].apply(lambda x:vessel_datasource[x])
## trace只有部分有
print(gps_record['TRANSPORT_TRACE'].isnull().sum())
print(gps_record.shape)
non_nan = gps_record['TRANSPORT_TRACE'].notna().tolist()
for index, b in enumerate(non_nan):
    if b:
        print(gps_record['TRANSPORT_TRACE'][index])

nextport na count: 431
nextport eta na count: 431
997
(1000, 13)
CNYTN-CLIQQ
CNYTN-CLIQQ
CNYTN-CLIQQ


In [15]:
# event data
events = set(loading_order_event['EVENT_CODE'].tolist())
event_code = dict(zip(events, range(len(events))))

{nan: 0, 'TRANSIT PORT ETD': 1, 'UPDATE SHIPMENT ETA': 2, 'TRANSIT PORT ETA': 3, 'SHIPMENT MIT INBOUND DATE': 4, 'ESTIMATED ARRIVAL TIME TO PORT': 5, 'TRANSIT PORT DECLARATION BEGIN': 6, 'ESTIMATED ARRIVAL TO PORT': 7, 'DISCHARGED': 8, 'ESTIMATED ONBOARD DATE': 9, 'PLANNED PICK UP DATE': 10, 'SHIPMENT ONBOARD DATE': 11, 'RDC ATD': 12, 'CONTAINER LOADED ON BOARD': 13, 'DAILY TRACK AND TRACE': 14, 'TRANSIT PORT ATA': 15, 'ARRIVAL AT CFS OR CY': 16, 'CARGO ARRIVAL AT DESTINATION': 17, 'IMP CUSTOMS CLEARANCE START': 18, 'IMP CUSTOMS CLEARANCE FINISHED': 19, 'TRANSIT PORT CUSTOMS RELEASE': 20, 'ARRIVAL AT PORT': 21, 'TRANSIT PORT ATD': 22, 'PICKED UP': 23}


对于GPS数据和事件数据之间的联系并不是很清楚，所以下面一部分先通过一定的方式把它们联系起来。

目前的发现有：

1. GPS的订单号loadingOrderEvent里面不一定有（比方说ZQ464072113491）
2. case study 如下
3. 

In [21]:
# 遍历GPS data
selected_lorder = 'XN821466065421'
print(gps_record[gps_record['loadingOrder'] == selected_lorder])
print(loading_order_event[loading_order_event['loadingOrder'] == selected_lorder])

loadingOrder carrierName                 timestamp   longtitue  \
113  XN821466065421      OIEQNT  2019-01-01T00:04:03.000Z  131.929728   
355  XN821466065421      OIEQNT  2019-01-01T00:07:52.000Z  131.949412   
566  XN821466065421      OIEQNT  2019-01-01T00:11:58.000Z  131.970683   
656  XN821466065421      OIEQNT  2019-01-01T00:16:04.000Z  131.991967   
744  XN821466065421      OIEQNT  2019-01-01T00:19:58.000Z  132.012213   
859  XN821466065421      OIEQNT  2019-01-01T00:24:10.000Z  132.033833   
933  XN821466065421      OIEQNT  2019-01-01T00:27:34.000Z  132.051342   
998  XN821466065421      OIEQNT  2019-01-01T00:31:40.000Z  132.072523   

      latitude   vesselMMSI  speed  direction vesselNextport  \
113  31.153467  J1826969247     32       7220     MANZANILLO   
355  31.160452  J1826969247     32       6650     MANZANILLO   
566  31.167910  J1826969247     32       6780     MANZANILLO   
656  31.175338  J1826969247     32       6840     MANZANILLO   
744  31.182527  J1826969247  

In [55]:
# pandas时间操作以及timestamp情况

print((test_data['loadingOrder'] == 'CF946210847851').sum())
print((test_data['loadingOrder'] == 'CI265639541482').sum())
TIMESTAMP_FORMAT = '%Y-%m-%dT%H:%M:%S.000Z'
test_time_delta = pd.to_datetime(test_data['timestamp'][:5], format=TIMESTAMP_FORMAT) - pd.to_datetime(test_data['timestamp'][:5],format=TIMESTAMP_FORMAT).shift()
print(test_time_delta.dt.total_seconds())   # 说明不是等长采样...

69
614
0       NaN
1    1020.0
2     480.0
3    2160.0
4    2760.0
Name: timestamp, dtype: float64
nan
1020
480
2160
2760


## Models
见[Ensembling & Stacking Models](https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python)一文，不过感觉这里面的处理不是很优雅，总之之后还可以改。

In [6]:
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)
    
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
    
    def predict(self, x):
        return self.clf.predict(x)

    def fit(self, x, y):
        self.clf.fit(x_train, y_train)
    
    def feature_importance(self, x, y):
        print(self.clf.fit(x, y).feature_importances_)